## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
# import wget
# !python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
# !tar -xvzf ml-20m-compact.tar.gz

In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])

unique_users = df['userId'].unique()
unique_items = df['movieId'].unique()

df

/tmp/ipykernel_850/381393729.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('./dataset/ratings_sample.csv')


,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [4]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [5]:

map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title

df

,userId,movieId,rating,title
0,0,0,5.0,Enemy Mine (1985)
1,0,1,4.5,Beautiful Thing (1996)
2,0,2,4.0,"Aristocats, The (1970)"
3,0,3,2.0,American Psycho (2000)
4,0,4,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,11089,22,5.0,Natural Born Killers (1994)
190617,11089,10,5.0,Dark City (1998)
190618,11089,2,4.0,"Aristocats, The (1970)"
190619,11089,21,4.5,Conspiracy Theory (1997)


In [6]:
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474


### Divisão da base em treino e teste

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [8]:
movies['movieId'] = movies['movieId'].map(map_items)

movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)
movies_genres.head()

,movieId,title,genre
0,49,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,49,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,61,Dangerous Minds (1995),Drama
2,125,Across the Sea of Time (1995),Documentary
2,125,Across the Sea of Time (1995),IMAX


Salvando a amostra de teste e de treino em um arquivo .dat para o Case Recommender usar

In [9]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

In [10]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', as_similar_first=True, similarity_metric='jaccard', k_neighbors=5).compute()


[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.504187 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.267401 sec
Eval:: MAE: 0.724223 RMSE: 0.951786 


### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [11]:
# !python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
# ! tar -xvzf ml-20m-features.tar.gz

In [12]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

No. movies: 433
Size of each word: 50


In [13]:
visualHistograms[8]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc. 

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima. 

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme. 

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações. 

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity). 

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

Salva os atributos dos itens em um arquivo chamado items_visual.dat, que sera usado pelo Case Recommender

## Assumindo que os vetores estão ordenados por movieId interno

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular similaridade por cosseno
similarity_matrix = cosine_similarity(visualHistograms)
df_similarity = pd.DataFrame(similarity_matrix)

# Salvar os dados num arquivo com o padrao itemId itemId similaridade 
similarity_list = df_similarity.stack().reset_index()
similarity_list.columns = ['itemId_1', 'itemId_2', 'similarity']


similarity_list.to_csv('visual_similarities.dat', index=False, header=False, sep='\t')

Usa o Case Recommender novamente, mas agora com uma nova métrica para prever os resultados

In [15]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN
#Por algum motivo o programa fala que encontrou o KeyError 417 indicando que nao achou referencia para o filme 417, no arquivo de notas, mas ele nem está
#na base de dados de treino ou teste

# Roda o modelo com as features visuais (matriz calculada)
# ItemAttributeKNN("train.dat", "test.dat", similarity_file="visual_similarities.dat", k_neighbors=5).compute()

***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos. 
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3. 
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados. 
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

Definindo relevância para o usuário e construção de dicionário

In [16]:
def get_user_interactions(user_id):
    user_train = train[train['userId'] == user_id].copy()
    return user_train

def evaluate_relevance(interactions_df):
    interactions_df['relevant'] = (interactions_df['rating'] >= 3).astype(int)
    return interactions_df

def get_genres(movieId):
    if movieId not in movies_genres['movieId'].values:
        return []
    return movies_genres.loc[(movies_genres.movieId==movieId),'genre'].tolist()

rel = evaluate_relevance(get_user_interactions(2))
rel

,userId,movieId,rating,title,relevant
37,2,28,2.0,My Best Friend's Wedding (1997),0
41,2,18,3.0,Life Is Beautiful (La Vita è bella) (1997),1
31,2,19,5.0,Reservoir Dogs (1992),1
29,2,12,4.0,Star Wars: Episode V - The Empire Strikes Back...,1
38,2,22,1.0,Natural Born Killers (1994),0
30,2,17,3.0,Chasing Amy (1997),1
40,2,20,4.0,While You Were Sleeping (1995),1
33,2,8,5.0,"Karate Kid, The (1984)",1
32,2,25,3.0,Keeping the Faith (2000),1
36,2,27,2.0,Zero Effect (1998),0


Treinamento do modelo com o Bayes e Laplace

In [17]:

from collections import Counter

def train_naive_bayes(userId):
    # Com base no id do usuário classificar filmes avaliados como relevantes ou nao
    user_data = evaluate_relevance(get_user_interactions(userId))

    # Probabilidade de ele achar qualquer filme relevante
    pRelevant = (user_data['relevant'] == 1).sum() / len(user_data)
    pNotRelevant = 1 - pRelevant

    relevant_movies = user_data[user_data['relevant'] == 1]
    notRelevant_movies = user_data[user_data['relevant'] == 0]

    # Quantidade de generos disponiveis
    V = movies_genres['genre'].nunique()
    all_genres = movies_genres['genre'].unique().tolist()

    # Generos que o usuário gosta
    relevant_movieId = relevant_movies['movieId'].tolist()
    relevant_genres = []
    for movieId in relevant_movieId:
        relevant_genres.extend(get_genres(movieId))

    notRelevant_movieId = notRelevant_movies['movieId'].tolist()
    notRelevant_genres = []
    for movieId in notRelevant_movieId:
        notRelevant_genres.extend(get_genres(movieId))

    n_relevant_genres = Counter(relevant_genres)
    n_notRelevant_genres = Counter(notRelevant_genres)

    # Calcular a probabilidade de gostar de cada genero
    likelihoods_relevant = {}
    likelihoods_notRelevant = {}
    count = 0

    for genre in all_genres:
        count = n_relevant_genres.get(genre, 0)
        probability = (count + 1) / (len(relevant_genres) + V)
        likelihoods_relevant[genre] = probability
    
    for genre in all_genres:
        count = n_notRelevant_genres.get(genre, 0)
        probability = (count + 1) / (len(notRelevant_genres) + V)
        likelihoods_notRelevant[genre] = probability

    model = {
            'priors': {'relevant': pRelevant, 'not_relevant': pNotRelevant},
            'likelihoods_relevant': likelihoods_relevant,
            'likelihoods_not_relevant': likelihoods_notRelevant
    }
    
    return model

train_naive_bayes(1)

{'priors': {'relevant': np.float64(1.0), 'not_relevant': np.float64(0.0)},
 'likelihoods_relevant': {'Crime': 0.06666666666666667,
  'Drama': 0.1111111111111111,
  'Documentary': 0.022222222222222223,
  'IMAX': 0.022222222222222223,
  'Thriller': 0.1111111111111111,
  'War': 0.06666666666666667,
  'Action': 0.06666666666666667,
  'Comedy': 0.08888888888888889,
  'Romance': 0.08888888888888889,
  'Animation': 0.022222222222222223,
  'Children': 0.022222222222222223,
  'Mystery': 0.06666666666666667,
  'Sci-Fi': 0.044444444444444446,
  'Adventure': 0.044444444444444446,
  'Fantasy': 0.022222222222222223,
  'Film-Noir': 0.022222222222222223,
  'Horror': 0.044444444444444446,
  'Western': 0.022222222222222223,
  'Musical': 0.022222222222222223,
  '(no genres listed)': 0.022222222222222223},
 'likelihoods_not_relevant': {'Crime': 0.05,
  'Drama': 0.05,
  'Documentary': 0.05,
  'IMAX': 0.05,
  'Thriller': 0.05,
  'War': 0.05,
  'Action': 0.05,
  'Comedy': 0.05,
  'Romance': 0.05,
  'Animatio

Realização das previsões

In [ ]:
import math

# O model no input é o retorno da função na célula de cima (resultado do treino)
def predict_relevance(model, movie_genres):
    #Operações logaritmicas
    score_relevant = math.log(model['priors']['relevant'])
    score_not_relevant = math.log(model['priors']['not_relevant'])

    for genre in movie_genres:
        if genre in model['likelihoods_relevant']:
            score_relevant += math.log(model['likelihoods_relevant'][genre])
        
        if genre in model['likelihoods_not_relevant']:
            score_not_relevant += math.log(model['likelihoods_not_relevant'][genre])
            
    #Predição
    if score_relevant > score_not_relevant:
        return "Relevant"
    else:
        return "Not Relevant"
    


 # Resultado - 74,30%

 Acurácia da previsão x valor real da avaliação

In [19]:
#conseguir todos os ids unicos dos usuários da base de teste
all_test_users = test['userId'].unique()

user_accuracies = []
users_evaluated = 0

# Testar para todos os usuários
for user_id in all_test_users:
    try:
        # Treinar modelo para esse usuário
        user_model = train_naive_bayes(user_id)

        user_test_data = test[test['userId'] == user_id]

        # Prever e comparar para cada filme
        if not user_test_data.empty:
            correct_predictions = 0
            
            # Percorrer por todos os filmes vistos do usuário
            for index, row in user_test_data.iterrows():
                movie_id = row['movieId']
                actual_rating = row['rating']
                
                # Relevancia real
                actual_label = "Relevant" if actual_rating >= 3 else "Not Relevant"
                
                # Extrair os generos do filme e prever rótulo
                movie_genres = get_genres(movie_id)
                prediction_label = predict_relevance(user_model, movie_genres)
                
                # Comparar previsçao com realidade
                if prediction_label == actual_label:
                    correct_predictions += 1

            # Calcular acurácia final
            total_predictions = len(user_test_data)
            accuracy = (correct_predictions / total_predictions) * 100
            user_accuracies.append(accuracy)
            users_evaluated += 1
            
            print(f"Acuracia final para o usuário {user_id}: {accuracy:.2f}%")
            print(f"({correct_predictions} de {total_predictions} previsões foram corretas.)")
            
    except Exception as e:
        # If training or prediction fails for a user, print a message and skip them
        print(f"Could not process User {user_id}. Skipping. Reason: {e}")

average_accuracy = np.mean(user_accuracies)
print("-" * 30)
print()
print(f"A média de acertos foi: {average_accuracy:.2f}%")


Acuracia final para o usuário 1836: 25.00%
(1 de 4 previsões foram corretas.)
Acuracia final para o usuário 8646: 60.00%
(3 de 5 previsões foram corretas.)
Could not process User 1464. Skipping. Reason: math domain error
Acuracia final para o usuário 5315: 72.73%
(8 de 11 previsões foram corretas.)
Acuracia final para o usuário 6571: 100.00%
(3 de 3 previsões foram corretas.)
Acuracia final para o usuário 126: 50.00%
(2 de 4 previsões foram corretas.)
Acuracia final para o usuário 1924: 37.50%
(3 de 8 previsões foram corretas.)
Acuracia final para o usuário 4376: 66.67%
(2 de 3 previsões foram corretas.)
Acuracia final para o usuário 2736: 10.00%
(1 de 10 previsões foram corretas.)
Acuracia final para o usuário 4549: 0.00%
(0 de 1 previsões foram corretas.)
Could not process User 5284. Skipping. Reason: math domain error
Acuracia final para o usuário 6546: 57.14%
(4 de 7 previsões foram corretas.)
Acuracia final para o usuário 788: 80.00%
(4 de 5 previsões foram corretas.)
Acuracia fin

 ***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier.

Preparar novos dados e binarizar as notas

In [35]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# Carregar os dados
new_movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', header=None)
new_ratings = pd.read_csv('ml-100k/u.data', sep='\t', header=None)

# Renomear colunas para facilitar o uso
new_movies.columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
new_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

# Manter apenas as colunas necessárias
new_movies = new_movies[['movie_id', 'title', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

# (>=3 relevante, <3 não relevante)
new_ratings['label'] = new_ratings['rating'].apply(lambda x: 1 if x >= 3 else 0)

new_ratings.head()

,user_id,movie_id,rating,timestamp,label
0,196,242,3,881250949,1
1,186,302,3,891717742,1
2,22,377,1,878887116,0
3,244,51,2,880606923,0
4,166,346,1,886397596,0


Dividir database

In [36]:
# Dividir as avaliações em treinamento e teste
train_ratings, test_ratings = train_test_split(new_ratings, test_size=0.2, random_state=42)


In [37]:

user_ratings = train_ratings[(train_ratings['user_id'] == 5)]
user_movies = pd.merge(user_ratings, new_movies, on='movie_id')
X = user_movies.drop(['user_id', 'movie_id', 'rating', 'timestamp', 'title'], axis=1)
y = user_movies['rating']
print(X)
print(y)

     label  Action  Adventure  Animation  Children  Comedy  Crime  \
0        1       0          0          0         0       1      0   
1        1       0          0          0         0       0      0   
2        0       0          0          0         0       1      0   
3        0       0          0          0         0       0      0   
4        1       0          0          0         0       0      0   
..     ...     ...        ...        ...       ...     ...    ...   
133      0       1          0          0         0       0      0   
134      0       0          0          0         0       0      0   
135      0       1          1          0         0       1      0   
136      0       0          0          0         0       0      0   
137      1       0          0          0         0       0      0   

     Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  \
0              0      0        0          0       0        0        0   
1              0      0  

Adaptando o modelo para ele prever notas de filmes não vistos e o treinando

In [39]:
# Inicializar objetos para armazenar os modelos
user_models = {}

# Para cada usuário, treinar um modelo
for user_id in train_ratings['user_id'].unique():
    # Selecionar amostra positiva
    user_rated_movies = new_ratings[new_ratings['user_id'] == user_id]
    positive_samples = user_rated_movies[['user_id', 'movie_id', 'label']]

    # Selecionar amostra negativa
    movies_seenIds = user_rated_movies['movie_id'].unique()
    all_movieIds = new_movies['movie_id'].unique()


    movies_notSeenIds = np.setdiff1d(all_movieIds, movies_seenIds)
    negative_movieIds = np.random.choice(movies_notSeenIds, size=len(positive_samples), replace=False)
    negative_samples = pd.DataFrame({'user_id': user_id, 'movie_id': negative_movieIds, 'label': 0 # Filmes não vistos não são relevantes
    })
    if len(movies_notSeenIds) < len(positive_samples):
        continue # Skip users with too few unseen movies to sample from

    # Combinando e preparando a amostra de treinamento completa 
    training_df = pd.concat([positive_samples, negative_samples], ignore_index=True)
        
    user_training_full = pd.merge(training_df, new_movies, on='movie_id')
    user_training_full = user_training_full.sample(frac=1, random_state=42)

    # Preparar os dados para treinamento da MLP
    x = user_training_full.drop(['user_id', 'movie_id', 'label', 'title'], axis=1)
    y = user_training_full['label']

    mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', max_iter=1000, random_state=42)
    mlp_classifier.fit(x, y)

    # Save the trained classifier for this user
    user_models[user_id] = mlp_classifier

In [55]:
def recomendar(user_id, k):
    if user_id in user_models:
        # Obter modelo dos usuários
        mlp = user_models[user_id]
        
        # Preparações
        user_rated_moviesIds = new_ratings[new_ratings['user_id'] == user_id]['movie_id']
        unseen_movies = new_movies[~new_movies['movie_id'].isin(user_rated_moviesIds)]
    
        X_unseen = unseen_movies.drop(['movie_id', 'title'], axis=1)
        
        # Prever avaliações para os filmes não visots
        relevance_probabilities = mlp.predict_proba(X_unseen)[:, 1]

        # Add the probabilities to our unseen_movies DataFrame
        unseen_movies['relevance_probability'] = relevance_probabilities

        recommended_movies = unseen_movies.sort_values(by='relevance_probability', ascending=False)

        return recommended_movies[['title', 'relevance_probability']].head(k)
    else:
        return f"No model available for User {user_id}."

In [59]:
target_user = 20

print(f"\nTop 10 recomendações {target_user} usando o MLPClassifier:")
top_movies_classifier = recomendar(target_user, k=10)
print(top_movies_classifier)



Top 10 recomendações 20 usando o MLPClassifier:
                                   title  relevance_probability
1238             Cutthroat Island (1995)               0.999345
91                   True Romance (1993)               0.999186
1609  Truth or Consequences, N.M. (1997)               0.999186
230                Batman Returns (1992)               0.999159
28                 Batman Forever (1995)               0.999159
172           Princess Bride, The (1987)               0.997986
384                     True Lies (1994)               0.997986
1218               Goofy Movie, A (1995)               0.996848
240     Last of the Mohicans, The (1992)               0.996780
464              Jungle Book, The (1994)               0.996053


/tmp/ipykernel_850/3867221464.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unseen_movies['relevance_probability'] = relevance_probabilities


Para um usuário qualquer, o MPLRegressor prevê a nota que o usuário daria, além de recomendar os filems com maior nota prevista. Isso provoca recomendações personalizadas. em contraposição o MPLClassifier prevê se o filme será relevante ou não (avaliado com mais de 3 estrelas), ambos os modelos são treinados por usuário, o que me faz acreditar que o modelo MPLRegressor é mais preciso